In [1]:
import os
import tempfile
import time
from datetime import datetime
import shutil
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [2]:
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET = os.environ.get("MINIO_BUCKET", "meteo-bronze")

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

✅ MinIO conectado


In [3]:
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
time.sleep(1)
conf = SparkConf().setAppName("Limpieza").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

✅ Spark iniciado


In [4]:
archivo_reciente = None
try:
    objects = minio_client.list_objects(MINIO_BUCKET, recursive=True)
    archivos_csv = [obj.object_name for obj in objects if obj.object_name.endswith(".csv")]
    if archivos_csv:
        archivo_reciente = sorted(archivos_csv)[-1]
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, archivo_reciente.split("/")[-1])
        minio_client.fget_object(MINIO_BUCKET, archivo_reciente, temp_file)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        print(f"✅ Cargados {df.count()} registros")
    else:
        archivo_reciente = "datos_ejemplo.csv"
        df = spark.createDataFrame([(1, 25.5)], ["id", "temp"])
except:
    archivo_reciente = "datos_ejemplo.csv"
    df = spark.createDataFrame([(1, 25.5)], ["id", "temp"])
print("✅ Datos cargados")

✅ Datos cargados


In [5]:
df = df.dropDuplicates()
df = df.drop('pressure', 'uv_level', 'pm25', 'rain_raw', 'wind_raw', 'vibration')
print(f"✅ {df.count()} registros limpios")

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Python313\Lib\socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\py4j\clientserver.py", line 538, in send_command
    

KeyboardInterrupt: 

In [ ]:
import csv
tabla = archivo_reciente.split('_bronce_')[0] if '_bronce_' in archivo_reciente else 'datos'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
archivo_silver = f'{tabla}_silver_{timestamp}.csv'
try:
    temp_file_local = os.path.join(tempfile.gettempdir(), archivo_silver)
    rows = df.collect()
    cols = df.columns
    with open(temp_file_local, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(cols)
        for row in rows:
            writer.writerow([str(row[c]) if row[c] is not None else '' for c in cols])
    minio_client.fput_object('meteo-silver', archivo_silver, temp_file_local)
    print(f'✅ {archivo_silver} guardado')
    os.remove(temp_file_local)
except Exception as e:
    print(f'❌ Error: {e}')
    import traceback; traceback.print_exc()